# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
1. [Introduction: Business Problem](#introduction)
2. [Data](#data)



## Introduction: Business Problem <a name="introduction"></a>

The objective of this project is to compare the neighbourhoods of two major cities: **London, the UK** and **Toronto, Canada**. In this project, I will focus on downtown Toronto and the western central London. By exploring the most common venues in each neighbourhood, I am trying to identify **the differences between the European and North American cities**, which may reflect *different city designs, lifestyles and cultures.*

This project might be interesting for:
* Students who want to study abroad in either North America or Europe
* Adults who are considering working abroad
* Travellers who are looking for their next destinations
* Researchers in the field of urban studies/human geography

## Data <a name="data"></a>

I will use the following datasets to collect the information needed for this project.


* The postal codes of western central London will be obtained from https://en.wikipedia.org/wiki/WC_postcode_area.
* The postal codes of downtown Toronto will be obtained from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.
* The geographical coordinates of each neighbourhood will be obtained using **Python Geocoder package**.
* The types and locations of venues in each neighborhood will be obtained using **Foursquare API**.